In [ ]:
# script to convert *.las files to potree 1.6 compatible octrees using laspy and numpy.
#
# Code is setup to enable parallel processing
# TODO: optimize code
# TODO: allow multicore processing
# TODO: allow more paramaters than just x,y,z to be parsed
# TODO: change code to module
# TODO: 

In [81]:
import os
import laspy 
import numpy as np
from pprint import pprint
import json

In [166]:
# A great source for *laz files is http://3dsm.bk.tudelft.nl/matahn 
# # convert laz to las
# # C:\Program Files (x86)\lastools\bin
# f = r'pointcloud_rotterdam_maas.laz'
# new_f = f.replace('.laz', '.las')
# cmd = r'"C:\Program Files (x86)\lastools\bin\las2las.exe" -i {} -o {} '.format(f,new_f)
# print(cmd)
# # call system command
# os.system(cmd)

"C:\Program Files (x86)\lastools\bin\las2las.exe" -i pointcloud_rotterdam_maas.laz -o pointcloud_rotterdam_maas.las 


0

In [167]:
lasfile = laspy.file.File(r'pointcloud_rotterdam_maas.las')

In [168]:
# http://pythonhosted.org/laspy/header.html
hdr = lasfile.header
print(hdr.scale, hdr.offset)
print(hdr.max)
print(hdr.min)

# define maximum bounding cube for data. MUST be cube, so all sides equal length
print(np.round((np.array(hdr.max) + np.array(hdr.min))/2))

[0.01, 0.01, 0.01] [0.0, 400000.0, 0.0]
[97475.19, 437763.27, 161.28]
[90728.32, 434483.92, -19.240000000000002]
[  9.41020000e+04   4.36124000e+05   7.10000000e+01]


In [169]:
CENTERS = {'x': 92074, 'y':437770, 'z':750}
SMALLEST_TILE_SIZE = 100 # meter
LEVELS =  7
SCALE = 0.01

# define size as nice power of two (not necessary)
SIZE = SMALLEST_TILE_SIZE * 2 ** LEVELS
BBOX = {k: np.array([v-SIZE/2, v+SIZE/2]) for k, v in CENTERS.items()}

print(BBOX)
# check if all points are within bounding box
print([h_min>=BBOX[dim][0] and h_max<=BBOX[dim][1] for
       dim, h_min, h_max in zip(['x','y','z'],hdr.min,hdr.max)])



{'x': array([ 85674.,  98474.]), 'y': array([ 431370.,  444170.]), 'z': array([-5650.,  7150.])}
[True, True, True]


In [170]:
# to avoid duplicating the point coordinates in memory, discretize the points based on their raw coordinates
# in unscaled integers

bbox_unscaled = {dim: ((BBOX[dim]-offset)/scale).astype('i4') for 
               dim, scale, offset in zip(['x','y','z'], hdr.scale, hdr.offset)}
print(bbox_unscaled)

{'x': array([8567400, 9847400]), 'y': array([3137000, 4417000]), 'z': array([-565000,  715000])}


In [171]:
# determine the bins to classify points
bins = {dim: np.linspace(bbox_unscaled[dim][0],
                         bbox_unscaled[dim][1],
                         2**(LEVELS)+1
                   ).astype('i4')[1:-1] for dim in ['x','y','z']}

In [173]:
# put all points in bins
binned = [np.digitize(getattr(lasfile,dim.upper()),bins[dim]) for dim in ['x','y','z']]

In [174]:
HIERACHY_STEP_SIZE = 4
def address_to_filename(address):
    if address is '':
        return os.path.join('test','r','r.bin')

    parts = [address[i:i+HIERACHY_STEP_SIZE] for i in range(0, len(address)+1, HIERACHY_STEP_SIZE)]
    parts[-1]='r' + address + '.bin'
    return os.path.join('test','r',*parts)

print(address_to_filename('000'))
print(address_to_filename('112312312'))  
print(address_to_filename('7000'))
print(address_to_filename(''))


test\r\r000.bin
test\r\1123\1231\r112312312.bin
test\r\7000\r7000.bin
test\r\r.bin


In [175]:
def address_2_origin(address):
    if address == '':
        address = '0'

    # return origin relative to bounding box origin in unscaled values
    level = len(address)
    bin_parts = [bin(int(c)+8)[3::] for c in address]
    x0 = int((int(''.join([b[0] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    y0 = int((int(''.join([b[1] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    z0 = int((int(''.join([b[2] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    return (x0, y0, z0)

print(*address_2_origin('000'))
print(*address_2_origin('112312312'))  
print(*address_2_origin('7000'))
print(*address_2_origin(''))

0 0 0
0 272500 1095000
640000 640000 640000
0 0 0


In [176]:
from pathlib import Path
p = Path('test/r')
for n in range(0,LEVELS-1):
    print('removed',len([f.unlink() for f in p.rglob('r'+'?' * (n) + '.bin')]),'files')


removed 1 files
removed 4 files
removed 4 files
removed 4 files
removed 4 files
removed 16 files


In [177]:
addresses = []
for x in set(binned[0]):
    p = np.equal(binned[0],x)
    subset_x = [b[p] for b in binned]
    address_x = bin(x + 2**LEVELS)[-LEVELS::]
    
    for y in set(subset_x[1]):
        q = np.equal(subset_x[1],y)
        subset_xy = [b[q] for b in subset_x]
        address_y = bin(y + 2**LEVELS)[-LEVELS::]
        
        for z in set(subset_xy[2]):
            r = np.equal(subset_xy[2],z)
            subset_xyz = [b[r] for b in subset_xy]
            address_z = bin(z + 2**LEVELS)[-LEVELS::]
            
            # make filename of 0-7's
            # https://github.com/potree/potree/blob/master/docs/file_format.md
            address = ''.join([str(int(''.join((x,y,z)),2)) for x,y,z in zip(address_x,address_y,address_z)])
            addresses.append(address)
            filename = address_to_filename(address)
            # print(filename)
            
            # extract relevant subset of data
            subset = np.where(p)[0][q][r]
            
            # convert points from int32 relative to hdr.scale and hdr.offset to uint32 relative to SCALE, BBOX
            # and the tile box address
            x0, y0, z0 = address_2_origin(address)
            
            if not os.path.exists(os.path.dirname(filename)):
                os.makedirs(os.path.dirname(filename))
            np.vstack((
                 (((lasfile.X[subset] * hdr.scale[0]) + hdr.offset[0] - BBOX['x'][0]) / SCALE).astype('<u4') - x0,
                 (((lasfile.Y[subset] * hdr.scale[1]) + hdr.offset[1] - BBOX['y'][0]) / SCALE).astype('<u4') - y0,
                 (((lasfile.Z[subset] * hdr.scale[2]) + hdr.offset[2] - BBOX['z'][0]) / SCALE).astype('<u4') - z0
                 )).transpose().tofile(filename)
            


In [178]:
# from pathlib import Path
import re
only_digits = re.compile(r"\D")

p = Path('test/r')
addresses = [only_digits.sub("",paths.parts[-1]) for paths in p.glob('**/r*.bin')]
print(addresses[::100])

['0573262', '0753444', '0755666', '0771442', '0775240', '0777464', '4137662', '4313040', '4315405', '4317466', '4331662', '4333626', '4337020', '4351240', '4353620', '4357226', '4371600', '4375042', '4377422', '4711600', '4715040', '4717440', '4733002', '4735402', '4751000', '4753426', '4773240']


In [179]:
# convert tree to tree
tree = {}
for a in addresses:
    t = tree
    for s in a:
        if s not in t:
            t[s] = {} 
        t = t[s]

In [180]:
# print(json.dumps(tree,sort_keys=True, indent=2, separators=(',', ': ')))
# convert to list of all possible tiles, and their children

def nodes_to_list(prefix, key, value):
    hrc = [{'address':prefix+key,
            'children': tuple(value.keys())}, 
          ]
    
    [hrc.extend(nodes_to_list(prefix+key,k,v)) for k, v in value.items()]
    
    return hrc

hrc = nodes_to_list('','',tree)
hrc.sort(key=lambda k: (len(k['address']), k['address']))


In [181]:
for item in hrc:
    filename = address_to_filename(item['address'])
    item.update(
        {
            'mask': sum([2 ** int(c) for c in item['children']]),
            'filename': filename,
            'directory': os.path.dirname(filename),
            'parentDirectory': os.path.dirname(address_to_filename(item['address'][:-1]))
        })
    
pprint(hrc[-5:-1])

[{'address': '6111602',
  'children': (),
  'directory': 'test\\r\\6111',
  'filename': 'test\\r\\6111\\r6111602.bin',
  'mask': 0,
  'parentDirectory': 'test\\r\\6111'},
 {'address': '6111604',
  'children': (),
  'directory': 'test\\r\\6111',
  'filename': 'test\\r\\6111\\r6111604.bin',
  'mask': 0,
  'parentDirectory': 'test\\r\\6111'},
 {'address': '6111606',
  'children': (),
  'directory': 'test\\r\\6111',
  'filename': 'test\\r\\6111\\r6111606.bin',
  'mask': 0,
  'parentDirectory': 'test\\r\\6111'},
 {'address': '6111640',
  'children': (),
  'directory': 'test\\r\\6111',
  'filename': 'test\\r\\6111\\r6111640.bin',
  'mask': 0,
  'parentDirectory': 'test\\r\\6111'}]


In [182]:
dt = np.dtype([('x','<u4'),('y','<u4'),('z','<u4')])
def read_file_at_address(address):
    points = np.fromfile(address_to_filename(address),dtype=dt)
    x0, y0, z0 = address_2_origin(address)
    points['x'] += x0
    points['y'] += y0
    points['z'] += z0
    return(points)

In [183]:
for h in hrc[::-1]:
    if not os.path.exists(h['filename']):
        points = np.hstack([read_file_at_address(h['address']+child) for child in h['children']])
        
        x0, y0, z0 = address_2_origin(h['address'])
        points['x'] -= x0
        points['y'] -= y0
        points['z'] -= z0

        points[::10].tofile(h['filename'])
    h['n_points'] = int(os.stat(h['filename']).st_size / dt.itemsize)
        

In [184]:
hrc_dtype = np.dtype([
    ("mask", "<u1"),            
    ("n_points", "<u4"),   
    ])

import re
only_digits = re.compile(r"\D")


for d in set(h['directory'] for h in hrc):
    hrc_data = np.array(
        [(h['mask'],h['n_points']) for h in hrc if 
         h['parentDirectory'] == d or h['directory'] == d],
        dtype=hrc_dtype)

    hrc_data.tofile(os.path.join(d, 'r' + only_digits.sub("",d) + '.hrc'))



In [185]:
options = {
    "version": "1.6",
    "octreeDir": "test",
    "boundingBox": {
        "lx": BBOX['x'][0],
        "ly": BBOX['y'][0],
        "lz": BBOX['z'][0],
        "ux": BBOX['x'][1],
        "uy": BBOX['y'][1],
        "uz": BBOX['z'][1]
    },
    "tightBoundingBox": {
        "lx": hdr.min[0],
        "ly": hdr.min[1],
        "lz": hdr.min[2],
        "ux": hdr.max[0],
        "uy": hdr.max[1],
        "uz": hdr.max[2]
    },
    "pointAttributes": ["POSITION_CARTESIAN"],
    "spacing": 0.01,
    "scale": SCALE,
    "hierarchyStepSize": HIERACHY_STEP_SIZE,
}
print(json.dumps(options,sort_keys=True, indent=2, separators=(',', ': ')))
with open('cloud.js','w') as f:
    json.dump(options,f)

{
  "boundingBox": {
    "lx": 85674.0,
    "ly": 431370.0,
    "lz": -5650.0,
    "ux": 98474.0,
    "uy": 444170.0,
    "uz": 7150.0
  },
  "hierarchyStepSize": 4,
  "octreeDir": "test",
  "pointAttributes": [
    "POSITION_CARTESIAN"
  ],
  "scale": 0.01,
  "spacing": 0.01,
  "tightBoundingBox": {
    "lx": 90728.32,
    "ly": 434483.92,
    "lz": -19.240000000000002,
    "ux": 97475.19,
    "uy": 437763.27,
    "uz": 161.28
  },
  "version": "1.6"
}
